# 3D Perception Tutorial

이 튜토리얼에서는 **PointCloud 데이터의 기초**부터 시작해,  
**3D semantic segmentation 모델을 직접 구현하고 실습**해봅니다.

실습은 **Google Colab 기반**으로 진행되며,  
**synthetic 데이터셋을 활용하여 빠르게 시각적 결과를 확인**할 수 있도록 구성되어 있습니다.

📅 제작일: 2025.09.09  
✍️ 작성자: 서민균

---

## 📚 목차

1. [3D 데이터 실습](#3d-데이터-실습)  
   - 포인트클라우드 시각화  
   - 포인트클라우드 가공 (다운샘플링, normal 추정, bounding box 등)  
   - Open3D + Plotly를 활용한 실습  

2. [3D Perception 실습](#3d-Perception-실습)   
   - Synthetic object에 대한 classification
   - Synthetic scene에 대한 semantic segmentation  
   - 시각화를 통한 결과 비교

---

## ⚠️ 주의사항

> 본 실습은 일부 모델 훈련 과정에서 GPU가 필요합니다.  
> 아래 경로를 따라 Colab 런타임 설정을 변경해 주세요:

**[런타임] → [런타임 유형 변경] → [하드웨어 가속기] → "GPU" 선택**

---

## 3D 데이터 실습

이번 실습에서는 3D 데이터 전처리와 시각화를 위해 **Open3D** 라이브러리를 사용합니다.

Open3D는 3D 데이터를 쉽게 다루기 위한 **Python/C++ 기반의 오픈소스 라이브러리**로,  
Point Cloud, Mesh, RGB-D 등 다양한 3D 형식을 지원하며  
**시각화, 정합(Registration), 재구성(Reconstruction), 필터링** 등 핵심 기능을 제공합니다.

<p>
  <img src="https://raw.githubusercontent.com/isl-org/Open3D/main/docs/_static/open3d_logo_horizontal.png" width="300"/>
</p>


🔗 [GitHub: isl-org/Open3D](https://github.com/isl-org/Open3D)

---

In [ ]:
!pip install open3d

In [ ]:
# --------------------------------------------------------
# 필수 라이브러리 import 및 환경 설정
# --------------------------------------------------------

import subprocess  # (필요 시) 시스템 명령 실행용
import sys         # Python 시스템 정보 확인용

# PyTorch 및 관련 모듈
import torch
import torch.nn as nn
import torch.nn.functional as F

# 수치 계산용 라이브러리
import numpy as np

# 경고 메시지 무시 (실습 시 깔끔한 출력용)
import warnings
warnings.filterwarnings('ignore')


# --------------------------------------------------------
# 현재 Python / PyTorch / CUDA 환경 정보 출력
# --------------------------------------------------------

# Python 버전
print(f"Python version: {sys.version}")

# PyTorch 버전
print(f"PyTorch version: {torch.__version__}")

# CUDA 사용 가능 여부 확인
print(f"CUDA available: {torch.cuda.is_available()}")

# CUDA가 사용 가능할 경우, 상세 정보 출력
if torch.cuda.is_available():
    print(f"CUDA version: {torch.version.cuda}")               # 설치된 CUDA 버전
    print(f"GPU: {torch.cuda.get_device_name(0)}")             # 첫 번째 GPU 이름

In [ ]:
# 시각화 및 그래픽 관련 라이브러리 불러오기
import matplotlib.pyplot as plt
import plotly
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

# GPU 사용 가능하면 CUDA, 아니면 CPU 사용
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# -----------------------------------------------------------------------------------
# 단일 포인트클라우드를 Plotly로 시각화하는 함수
# -----------------------------------------------------------------------------------
def visualize_point_cloud(points, colors=None, title="Point Cloud",
                         point_size=2, colorscale='Viridis', show_axis=True):
    """
    Plotly를 이용한 3D 포인트클라우드 인터랙티브 시각화

    Args:
        points: (N, 3) numpy 또는 torch.Tensor 형태의 점 좌표
        colors: 점 색상 (RGB 또는 스칼라 값)
        title: 플롯 제목
        point_size: 점의 크기
        colorscale: Plotly의 색상맵 이름
        show_axis: 축 이름 표시 여부
    """
    # Tensor → NumPy 변환
    if isinstance(points, torch.Tensor):
        points = points.cpu().numpy()

    # 색상이 없으면 z값 기반 색상 자동 설정
    if colors is None:
        colors = points[:, 2]
    elif isinstance(colors, torch.Tensor):
        colors = colors.cpu().numpy()

    # Plotly 3D scatter 생성
    fig = go.Figure(data=[go.Scatter3d(
        x=points[:, 0],
        y=points[:, 1],
        z=points[:, 2],
        mode='markers',
        marker=dict(
            size=point_size,
            color=colors if len(colors.shape) == 1 else colors[:, 0],  # RGB라면 첫 채널만 사용
            colorscale=colorscale,
            showscale=True,
            colorbar=dict(title="Value", thickness=20, len=0.7)
        ),
        # Hover text (최대 1000개만 표시)
        text=[f"Point {i}<br>x: {x:.3f}<br>y: {y:.3f}<br>z: {z:.3f}"
              for i, (x, y, z) in enumerate(points[:min(1000, len(points))])],
        hovertemplate='%{text}<extra></extra>'
    )])

    # 시각화 레이아웃 설정
    fig.update_layout(
        title=dict(text=title, x=0.5, xanchor='center'),
        scene=dict(
            xaxis=dict(title='X' if show_axis else '', showgrid=True),
            yaxis=dict(title='Y' if show_axis else '', showgrid=True),
            zaxis=dict(title='Z' if show_axis else '', showgrid=True),
            camera=dict(eye=dict(x=1.5, y=1.5, z=1.5)),
            aspectmode='auto'
        ),
        width=900,
        height=700,
        margin=dict(r=20, b=10, l=10, t=40),
        showlegend=False
    )

    fig.show()

# -----------------------------------------------------------------------------------
# 여러 개의 포인트클라우드를 같은 화면에 시각화하는 함수
# -----------------------------------------------------------------------------------
def visualize_multiple_point_clouds(point_clouds_dict, title="Multiple Point Clouds"):
    """
    여러 포인트클라우드를 각각 다른 색상으로 한 플롯에 시각화

    Args:
        point_clouds_dict: {이름: (N,3) 포인트배열} 형태의 딕셔너리
        title: 플롯 제목
    """
    fig = go.Figure()
    colors = px.colors.qualitative.Set1  # 카테고리별 색상 팔레트

    for idx, (name, points) in enumerate(point_clouds_dict.items()):
        if isinstance(points, torch.Tensor):
            points = points.cpu().numpy()

        color = colors[idx % len(colors)]

        fig.add_trace(go.Scatter3d(
            x=points[:, 0],
            y=points[:, 1],
            z=points[:, 2],
            mode='markers',
            name=name,
            marker=dict(size=3, color=color, opacity=0.8),
            text=[f"{name} - Point {i}" for i in range(min(100, len(points)))],
            hovertemplate='%{text}<br>x: %{x:.3f}<br>y: %{y:.3f}<br>z: %{z:.3f}<extra></extra>'
        ))

    fig.update_layout(
        title=dict(text=title, x=0.5, xanchor='center'),
        scene=dict(
            xaxis_title='X',
            yaxis_title='Y',
            zaxis_title='Z',
            camera=dict(eye=dict(x=1.5, y=1.5, z=1.5))
        ),
        width=1000,
        height=700,
        showlegend=True,
        legend=dict(x=0.02, y=0.98, bgcolor='rgba(255,255,255,0.8)')
    )

    fig.show()

# -----------------------------------------------------------------------------------
# 샘플 포인트클라우드 생성 함수 (sphere, cube, torus 등)
# -----------------------------------------------------------------------------------
def generate_sample_point_cloud(n_points=1000, shape='sphere', noise_level=0.0):
    """
    기본적인 3D 도형 기반 synthetic point cloud 생성

    Args:
        n_points: 생성할 점의 개수
        shape: 도형 종류 ('sphere', 'cube', 'torus', 'cylinder')
        noise_level: 가우시안 노이즈 크기
    Returns:
        points: (N,3) ndarray (x, y, z 좌표)
        features: (N,3) RGB 색상값 (0~1 랜덤)
    """
    if shape == 'sphere':
        # 🌐 구 표면 상의 점 생성
        # (theta, phi): 구면 좌표계 (longitude, latitude)
        theta = np.random.uniform(0, 2*np.pi, n_points)       # 경도 (longitude)
        phi   = np.random.uniform(0, np.pi, n_points)         # 위도 (latitude)
        r     = np.random.uniform(0.8, 1.2, n_points)         # 반지름의 약간의 무작위성

        # 구면 좌표계 → 직교 좌표계로 변환
        x = r * np.sin(phi) * np.cos(theta)  # x축
        y = r * np.sin(phi) * np.sin(theta)  # y축
        z = r * np.cos(phi)                  # z축 (위도 중심)

    elif shape == 'cube':
        # 🔲 정육면체 내부 점 생성
        # 각 축별로 [-1, 1] 범위에서 균등 샘플링
        x = np.random.uniform(-1, 1, n_points)
        y = np.random.uniform(-1, 1, n_points)
        z = np.random.uniform(-1, 1, n_points)

    elif shape == 'torus':
        # 🍩 도넛형 토러스 표면 점 생성
        # (theta: 주축 회전, phi: 단면 회전)
        theta = np.random.uniform(0, 2*np.pi, n_points)  # 도넛 중심 축을 따라 회전
        phi   = np.random.uniform(0, 2*np.pi, n_points)  # 단면(원) 내에서 회전
        R, r  = 1.0, 0.3                                 # R: 큰 반지름 (도넛 중심에서 원까지), r: 단면 원의 반지름

        # 토러스의 파라메트릭 방정식
        x = (R + r * np.cos(phi)) * np.cos(theta)  # 원을 x-y 평면 위에서 회전시킨 구조
        y = (R + r * np.cos(phi)) * np.sin(theta)
        z = r * np.sin(phi)

    elif shape == 'cylinder':
        # 🧱 원기둥 표면 점 생성
        theta = np.random.uniform(0, 2*np.pi, n_points)    # 원기둥 측면의 회전각
        z     = np.random.uniform(-1, 1, n_points)         # 높이 (축방향)
        r     = np.random.uniform(0.8, 1.0, n_points)      # 반지름 조절

        # 원기둥의 바깥 표면 점 (측면 기준)
        x = r * np.cos(theta)
        y = r * np.sin(theta)

    else:
        raise ValueError(f"Unknown shape: {shape}")

    # 점 좌표 (N, 3)
    points = np.stack([x, y, z], axis=1).astype(np.float32)

    # 노이즈 추가 (가우시안 잡음)
    if noise_level > 0:
        noise = np.random.randn(*points.shape) * noise_level
        points += noise.astype(np.float32)

    # RGB 색상값 (랜덤)
    features = np.random.rand(n_points, 3).astype(np.float32)

    return points, features

In [ ]:
# 시각화할 도형 리스트
shapes = ['sphere', 'cube', 'torus', 'cylinder']
point_clouds = {}

# 각 도형마다 샘플 포인트클라우드를 생성하고 딕셔너리에 저장
for shape in shapes:
    points, features = generate_sample_point_cloud(2000, shape)
    point_clouds[shape] = points  # features는 여기선 사용하지 않음

# ------------------------------------------------------------
# Sphere 도형만 따로 5000 포인트로 생성하여 단독 시각화
# ------------------------------------------------------------
points_sphere, features_sphere = generate_sample_point_cloud(5000, 'sphere', noise_level=0.02)
print(f"Points shape: {points_sphere.shape}")    # (5000, 3)
print(f"Features shape: {features_sphere.shape}")  # (5000, 3) → RGB 랜덤 색상

# z좌표(높이)를 기반으로 색상을 입혀서 인터랙티브하게 시각화
fig = visualize_point_cloud(
    points_sphere,
    colors=points_sphere[:, 2],  # z축 값에 따라 색상 표현
    title="Interactive Sphere Point Cloud (5000 points)",
    point_size=2,
    colorscale='Viridis'  # 색상맵: Viridis
)

# ------------------------------------------------------------
# 4가지 도형을 한 화면에서 비교 시각화
# 각 도형은 서로 다른 색상으로 표현됨
# ------------------------------------------------------------
visualize_multiple_point_clouds(point_clouds, title="Comparison of Different Point Cloud Shapes")

In [ ]:
import open3d as o3d
import numpy as np
import plotly.graph_objects as go

def plot_pointcloud_plotly(pcd, point_size=1.5, title=None, background='white'):
    """
    Plotly를 활용하여 Open3D PointCloud 객체를 시각화합니다.

    Args:
        pcd (o3d.geometry.PointCloud): Open3D 포인트클라우드 객체
        point_size (float): 점 크기
        title (str): 시각화 타이틀 (선택)
        background (str): 배경 색상 ('white', 'black' 등)

    Returns:
        plotly.graph_objects.Figure (현재는 show만 하고 반환은 주석 처리됨)
    """
    # 유효한 Open3D PointCloud 객체인지 확인
    if not isinstance(pcd, o3d.geometry.PointCloud):
        raise TypeError("Input must be an Open3D PointCloud")

    # 포인트 좌표 추출
    points = np.asarray(pcd.points)
    if points.shape[0] == 0:
        raise ValueError("Point cloud is empty")

    # 색상 정보가 있다면 추출 (없으면 None)
    colors = np.asarray(pcd.colors) if pcd.has_colors() else None

    # Plotly의 3D Scatter 그래프 구성
    fig = go.Figure(data=[go.Scatter3d(
        x=points[:, 0], y=points[:, 1], z=points[:, 2],
        mode='markers',
        marker=dict(
            size=point_size,
            color=colors if colors is not None else 'blue',  # 색상이 없으면 파란색으로
            opacity=1.0
        )
    )])

    # 레이아웃 설정
    fig.update_layout(
        scene=dict(
            xaxis=dict(visible=False),
            yaxis=dict(visible=False),
            zaxis=dict(visible=False),
            bgcolor=background  # 배경 색상
        ),
        margin=dict(r=0, l=0, b=0, t=40 if title else 0),
        title=title or ""
    )

    # 화면에 표시
    fig.show()

    # 필요하면 반환값으로 fig도 리턴 가능 (현재는 주석처리)
    # return fig

In [ ]:
# Open3D에서 제공하는 샘플 PLY 포인트클라우드 데이터를 불러옵니다
sample_ply_data = o3d.data.PLYPointCloud()

# PLY 파일을 읽어서 Open3D 포인트클라우드 객체로 변환합니다
pcd = o3d.io.read_point_cloud(sample_ply_data.path)

# Plotly를 이용해 포인트클라우드를 3D로 시각화합니다
plot_pointcloud_plotly(pcd, point_size=1.5, title="Open3D Sample")

# ------------------------------------------------------------------------
# 전체 PointCloud의 스케일(범위)를 계산하여, voxel_size 설정에 참고
# ------------------------------------------------------------------------

points_np = np.asarray(pcd.points)

# 각 축별 min/max 계산 → 전체 bounding box의 크기 추정
mins = points_np.min(axis=0)
maxs = points_np.max(axis=0)
extent = maxs - mins  # 각 축 방향의 전체 길이 (단위: 미터)

print(f"[PointCloud 범위] X: {extent[0]:.3f} m, Y: {extent[1]:.3f} m, Z: {extent[2]:.3f} m")
print(f"→ 전체 대각선 길이 (bounding box): {np.linalg.norm(extent):.3f} m")

In [ ]:
# ------------------------------------------------------------------------
# Open3D PointCloud 객체의 주요 속성 출력
# ------------------------------------------------------------------------

print("📌 [PointCloud 기본 정보]")
print(f"총 포인트 수          : {len(pcd.points):,} points")
print(f"컬러 정보 포함 여부   : {pcd.has_colors()}")
print(f"노멀 정보 포함 여부   : {pcd.has_normals()}")

print("\n📐 [기하 정보]")
print(f"중심 좌표              : {pcd.get_center()}")
print(f"최소 좌표 (min bound) : {pcd.get_min_bound()}")
print(f"최대 좌표 (max bound) : {pcd.get_max_bound()}")

# Bounding box 크기 출력
aabb = pcd.get_axis_aligned_bounding_box()
extent = aabb.get_extent()
print(f"축 정렬 AABB 크기     : {extent[0]:.3f} × {extent[1]:.3f} × {extent[2]:.3f} m³")

In [ ]:
# 원본 포인트 수 출력
num_points_original = len(pcd.points)
print(f"\n[원본 포인트 수] {num_points_original:,} points")

# ------------------------------------------------------------------------
# 다운샘플링에 사용할 voxel 크기 설정 (단위: 미터)
# ------------------------------------------------------------------------

voxel_size = 0.05  # 0.05m = 5cm 격자 크기

print(f"\n[Downsampling] Voxel size: {voxel_size} m")

# ------------------------------------------------------------------------
# [📌 Voxel-based Downsampling 설명 생략 (이미 있음)]
# ------------------------------------------------------------------------

# 실제 다운샘플링 실행
downpcd = pcd.voxel_down_sample(voxel_size=voxel_size)

# 다운샘플된 포인트 수 출력
num_points_down = len(downpcd.points)
print(f"[다운샘플 후 포인트 수] {num_points_down:,} points")

# 절감률 출력
reduction_ratio = 100 * (1 - num_points_down / num_points_original)
print(f"→ {reduction_ratio:.1f}% 포인트 감소")

# ------------------------------------------------------------------------
# Plotly 시각화: 다운샘플된 결과를 인터랙티브하게 시각화
# ------------------------------------------------------------------------
plot_pointcloud_plotly(
    downpcd,
    point_size=1.5,
    title=f"Downsampled (voxel={voxel_size})"
)

In [ ]:
import time  # 실행 시간 측정용

# ------------------------------------------------------------------------
# 선택할 포인트 수 설정 (예: 2000개)
# ------------------------------------------------------------------------
n_samples = 2000

print(f"Downsample the point cloud by selecting {n_samples} farthest points.")

# ------------------------------------------------------------------------
# [📌 Farthest Point Sampling (FPS)]

# - 포인트클라우드 전체에서 분포가 넓게 퍼진 대표 포인트들을 선택하는 방법
# - 과정:
#     1. 무작위로 시작점 하나를 선택
#     2. 현재 선택된 점들로부터 가장 먼 점을 추가
#     3. 이 과정을 원하는 샘플 수까지 반복
# - 결과적으로:
#     • 전체 공간을 고르게 커버하는 점들을 선택
#     • 형태 보존 (shape preservation)에 매우 유리
#     • 학습 데이터 전처리로 자주 사용됨 (PointNet, PointNet++, DGCNN 등에서)
# - 단점:
#     • 계산량이 비교적 큼 (Open3D는 최적화되어 빠른 편)
# ------------------------------------------------------------------------

# 실행 시간 측정 시작
start_time = time.time()

# FPS 실행
downpcd_farthest = pcd.farthest_point_down_sample(n_samples)

# 실행 시간 측정 종료
elapsed_time = time.time() - start_time

# 샘플링 후 포인트 수 출력
print("Points:", np.asarray(downpcd_farthest.points).shape)
print(f"⏱️ FPS 실행 시간: {elapsed_time:.3f}초")

# ------------------------------------------------------------------------
# Plotly 시각화
# ------------------------------------------------------------------------
plot_pointcloud_plotly(
    downpcd_farthest,
    point_size=1.5,
    title=f"Farthest Downsampled ({n_samples} pts)"
)

In [ ]:
# ------------------------------------------------------------------------
# [📌 Polygon Volume을 이용한 포인트클라우드 영역 잘라내기 (Crop)]
# ------------------------------------------------------------------------

# 데모용 polygon volume + pointcloud 경로 불러오기
# - point_cloud.ply: 전체 포인트클라우드
# - cropped.json: polygon volume 정의 (2D 다각형 + 높이 범위)
print("Load a polygon volume and use it to crop the original point cloud")
demo_crop_data = o3d.data.DemoCropPointCloud()

# 전체 포인트클라우드 로딩
pcd = o3d.io.read_point_cloud(demo_crop_data.point_cloud_path)

# JSON 형식의 2D 다각형 selection volume 로딩
# - z 범위 + 평면 상의 polygon으로 구성된 Volume
vol = o3d.visualization.read_selection_polygon_volume(demo_crop_data.cropped_json_path)

# 선택된 Volume 안에 있는 점들만 crop
# - 내부적으로 axis-aligned bounding box + polygon mask를 적용함
chair = vol.crop_point_cloud(pcd)

# 시각화 (Plotly 기반)
plot_pointcloud_plotly(
    chair,
    point_size=1.5,
    title="chair"
)

In [ ]:
# ------------------------------------------------------------------------
# 포인트클라우드에 동일한 색상 입히기 (Paint Uniform Color)
# ------------------------------------------------------------------------

print("Paint chair")

# 모든 점에 동일한 RGB 색상을 부여
# 여기서는 밝은 주황색 ([R,G,B] = [1.0, 0.706, 0.0]) 으로 설정
chair.paint_uniform_color([1, 0.706, 0])

# 시각화
plot_pointcloud_plotly(
    chair,
    point_size=1.5,
    title="chair"
)

In [ ]:
# ------------------------------------------------------------------------
# 전체 포인트클라우드에서 의자(chair) 부분만 제거하기
# ------------------------------------------------------------------------

# 전체 포인트클라우드 및 crop 영역 로드
demo_crop_data = o3d.data.DemoCropPointCloud()
pcd = o3d.io.read_point_cloud(demo_crop_data.point_cloud_path)

# polygon volume 정의 (의자 부분)
vol = o3d.visualization.read_selection_polygon_volume(demo_crop_data.cropped_json_path)

# 의자 영역 추출 (crop)
chair = vol.crop_point_cloud(pcd)

# ------------------------------------------------------------------------
# [📌 의자 부분 제거하기]

# 전체 포인트(pcd)에서 의자 포인트(chair)까지의 거리 계산
# → 각 점마다 chair와의 최단 거리
dists = pcd.compute_point_cloud_distance(chair)
dists = np.asarray(dists)

# 거리 > 0.01m인 점만 선택 (즉, 의자와 충분히 떨어진 점들만 유지)
ind = np.where(dists > 0.01)[0]

# 선택된 인덱스만 추출 → 의자 제외한 포인트클라우드
pcd_without_chair = pcd.select_by_index(ind)

# ------------------------------------------------------------------------
# Plotly로 시각화: 의자가 제거된 포인트클라우드
# ------------------------------------------------------------------------
plot_pointcloud_plotly(
    pcd_without_chair,
    point_size=1.5,
    title="pcd_without_chair"
)

## 3D Perception 실습

이 실습에서는 synthetic 3D point cloud 데이터를 활용하여 다음 두 가지 핵심 3D 인식 과제를 직접 수행해봅니다:

	1.	Point Cloud Classification
	•	입력된 3D 점군이 어떤 객체(예: 구, 큐브)인지 구분합니다.
	2.	Point-wise Segmentation
	•	한 장면 내 각 포인트가 어떤 구성 요소(예: 바닥, 벽, 박스)에 속하는지 예측합니다.

✅ 모든 실습은 가볍고 빠른 synthetic 데이터 기반으로 진행됩니다.

이는 복잡한 전처리나 대용량 파일 다운로드 없이, Google Colab 상에서 즉시 학습 및 시각화 결과를 확인할 수 있도록 설계된 것입니다.

In [ ]:
# ------------------------------------------------------------------------
# 필수 모듈 import
# ------------------------------------------------------------------------
import os, math, random, numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt

# 시드 고정 → 실험 재현성을 위해 중요
torch.manual_seed(0)
np.random.seed(0)
random.seed(0)

# 디바이스 설정 (CUDA 사용 가능 시 GPU로)
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Device:", device)


# ------------------------------------------------------------------------
# [유틸 함수] → 포인트클라우드를 이미지로 저장 (Open3D 없이)
# ------------------------------------------------------------------------
def save_pc_png(points, colors=None, fname="pc.png", elev=20, azim=60, s=1, title=None):
    """
    matplotlib을 이용해 3D 포인트클라우드를 시각화하고 PNG로 저장하는 함수
    """
    pts = points
    if colors is None:
        colors = np.full((pts.shape[0], 3), 0.35)  # 기본 회색
    fig = plt.figure(figsize=(5,5))
    ax = fig.add_subplot(111, projection='3d')
    ax.view_init(elev=elev, azim=azim)  # 카메라 시점 조절
    ax.scatter(pts[:,0], pts[:,1], pts[:,2], s=s, c=colors)
    ax.set_axis_off()
    if title: plt.title(title)

    # 비율 맞춤 (equal aspect)
    mins = pts.min(0); maxs = pts.max(0)
    ctr = (mins+maxs)/2; rng = (maxs-mins).max()/2
    ax.set_xlim(ctr[0]-rng, ctr[0]+rng)
    ax.set_ylim(ctr[1]-rng, ctr[1]+rng)
    ax.set_zlim(ctr[2]-rng, ctr[2]+rng)

    plt.tight_layout()
    plt.savefig(fname, dpi=200, bbox_inches='tight', pad_inches=0)
    plt.close()

In [ ]:
# ------------------------------------------------------------------------
# [📌 sample_sphere] 구(sphere) 표면의 점들을 샘플링하는 함수
# ------------------------------------------------------------------------
def sample_sphere(n=1024, r=1.0):
    """
    구 표면 위에 점을 균일하게 샘플링한 후, 노이즈를 추가하여 생성

    Args:
        n: 생성할 점의 수
        r: 구의 반지름

    Returns:
        (n, 3) 형태의 numpy 배열 (x, y, z)
    """
    phi = np.random.rand(n) * 2 * np.pi       # 0 ~ 2pi 사이의 방위각 (azimuth)
    costh = np.random.rand(n) * 2 - 1         # -1 ~ 1 사이의 cos(theta)
    theta = np.arccos(costh)                  # 극각 (theta), 위/아래 방향 조절

    # 구 표면의 x, y, z 좌표 계산 (구면 좌표 → 직교 좌표)
    x = r * np.sin(theta) * np.cos(phi)
    y = r * np.sin(theta) * np.sin(phi)
    z = r * np.cos(theta)

    pts = np.stack([x, y, z], axis=1).astype(np.float32)

    # 노이즈 추가 (정규분포)
    pts += 0.02 * np.random.randn(n, 3).astype(np.float32)
    return pts


# ------------------------------------------------------------------------
# [📌 sample_cube] 정육면체(cube) 표면 위에 점을 샘플링하는 함수
# ------------------------------------------------------------------------
def sample_cube(n=1024, s=2.0):
    """
    정육면체의 6개 면 위에 점을 랜덤하게 샘플링한 후, 노이즈 추가

    Args:
        n: 생성할 점 수
        s: 큐브의 한 변 길이 (전체 크기)

    Returns:
        (n, 3) 형태의 numpy 배열
    """
    # -0.5 ~ 0.5 사이 랜덤값 생성 후, 크기 스케일 적용 → (-s/2 ~ s/2)
    pts = (np.random.rand(n, 3).astype(np.float32) - 0.5) * s

    # 각 점마다 랜덤하게 한 면(face)을 선택 (총 6면)
    face = np.random.randint(0, 6, size=(n,))  # 0~5

    for i, f in enumerate(face):
        a = (s / 2) if f % 2 == 0 else -(s / 2)  # 양쪽 면 위치
        axis = f // 2                           # 0: x축, 1: y축, 2: z축
        pts[i, axis] = a + 0.02 * np.random.randn()  # 선택된 면으로 이동 + 노이즈

    # 전체에 약간의 노이즈 추가
    pts += 0.02 * np.random.randn(n, 3).astype(np.float32)
    return pts

In [ ]:
# ------------------------------------------------------------------------
# 두 개의 샘플 point cloud 생성 (Sphere vs Cube)
# ------------------------------------------------------------------------
pts_sphere = sample_sphere(n=1024)  # 구 표면 점들 생성
pts_cube   = sample_cube(n=1024)    # 정육면체 면 점들 생성


# ------------------------------------------------------------------------
# [📌 시각화 함수] 두 개의 point cloud를 나란히 3D로 시각화 (matplotlib)
# ------------------------------------------------------------------------
def viz_pc(pts1, pts2,
           titles=["Sphere", "Cube"],
           fname="sphere_cube_grid.png",
           elev=20, azim=60, s=0.5, figsize=(6,3)):
    """
    두 개의 point cloud를 나란히 3D 시각화하여 저장하는 함수

    Args:
        pts1, pts2: (N,3) 형태의 포인트클라우드 (numpy array)
        titles: 각 subplot의 제목 리스트
        fname: 저장할 파일 이름
        elev, azim: 카메라 시점 (elevation, azimuth)
        s: 점 크기 (scatter size)
        figsize: 전체 figure 크기 (width, height)
    """
    fig = plt.figure(figsize=figsize)

    for i, (pts, title) in enumerate(zip([pts1, pts2], titles)):
        ax = fig.add_subplot(1, 2, i+1, projection='3d')
        ax.view_init(elev=elev, azim=azim)  # 카메라 시점 설정
        ax.scatter(pts[:,0], pts[:,1], pts[:,2], s=s, c='dimgray')  # 점 시각화
        ax.set_title(title, fontsize=10)
        ax.set_axis_off()

        # 비율을 맞추기 위한 설정 (equal aspect)
        mins = pts.min(0)
        maxs = pts.max(0)
        ctr = (mins + maxs) / 2
        rng = (maxs - mins).max() / 2
        ax.set_xlim(ctr[0] - rng, ctr[0] + rng)
        ax.set_ylim(ctr[1] - rng, ctr[1] + rng)
        ax.set_zlim(ctr[2] - rng, ctr[2] + rng)

    # 저장 또는 화면 출력
    plt.tight_layout()
    plt.savefig(fname, dpi=200, bbox_inches='tight', pad_inches=0)
    plt.show()


# ------------------------------------------------------------------------
# 실행 및 이미지 저장
# ------------------------------------------------------------------------
viz_pc(pts_sphere, pts_cube)

In [ ]:
# ------------------------------------------------------------------------
# [PointNet 블록 1] STN3d (Spatial Transformer Network)
# → 입력 포인트(3D 좌표)에 대한 정렬/변환 학습 (예: 회전 보정)
# ------------------------------------------------------------------------
class STN3d(nn.Module):
    def __init__(self):
        super().__init__()
        # (B,3,N) → (B,1024) → affine 3x3 행렬 예측
        self.mlp1 = nn.Sequential(
            nn.Conv1d(3,64,1), nn.BatchNorm1d(64), nn.ReLU(True),
            nn.Conv1d(64,128,1), nn.BatchNorm1d(128), nn.ReLU(True),
            nn.Conv1d(128,1024,1), nn.BatchNorm1d(1024), nn.ReLU(True)
        )
        self.fc = nn.Sequential(
            nn.Linear(1024,512), nn.BatchNorm1d(512), nn.ReLU(True),
            nn.Linear(512,256), nn.BatchNorm1d(256), nn.ReLU(True),
            nn.Linear(256,9)  # 3x3 행렬 flatten
        )

    def forward(self, x):  # x: (B,3,N)
        b,_,n = x.shape
        f = self.mlp1(x).max(-1).values     # Global max-pool → (B,1024)
        t = self.fc(f)                      # (B,9)
        idt = torch.eye(3, device=x.device).view(1,9).repeat(b,1)  # 단위 행렬
        t = t + idt                         # 초기값은 항등변환
        t = t.view(-1,3,3)                  # (B,3,3)
        x = torch.bmm(t, x)                 # 입력 좌표에 변환 적용
        return x


# ------------------------------------------------------------------------
# [PointNet 블록 2] 포인트 피처 추출기 (global or per-point)
# ------------------------------------------------------------------------
class PointNetFeat(nn.Module):
    def __init__(self, global_feat=True, feat_dim=1024):
        super().__init__()
        self.stn = STN3d()
        self.conv1 = nn.Sequential(nn.Conv1d(3,64,1), nn.BatchNorm1d(64), nn.ReLU(True))
        self.conv2 = nn.Sequential(nn.Conv1d(64,128,1), nn.BatchNorm1d(128), nn.ReLU(True))
        self.conv3 = nn.Sequential(nn.Conv1d(128,feat_dim,1), nn.BatchNorm1d(feat_dim), nn.ReLU(True))
        self.global_feat = global_feat

    def forward(self, x):  # x: (B,3,N)
        x = self.stn(x)
        x = self.conv1(x); x = self.conv2(x); x = self.conv3(x)  # (B,F,N)

        if self.global_feat:
            return x.max(-1).values   # Global feature만 반환 (B,F)
        else:
            g = x.max(-1, keepdim=True).values
            x = torch.cat([x, g.repeat(1,1,x.shape[-1])], dim=1)  # (B,2F,N)
            return x


# ------------------------------------------------------------------------
# [PointNet Classification Head]
# - PointNet global feature → 분류기
# - 예: sphere vs cube vs chair
# ------------------------------------------------------------------------
class PointNetCls(nn.Module):
    def __init__(self, num_classes=2, feat_dim=1024):
        super().__init__()
        self.feat = PointNetFeat(global_feat=True, feat_dim=feat_dim)
        self.fc = nn.Sequential(
            nn.Linear(feat_dim,512), nn.BatchNorm1d(512), nn.ReLU(True), nn.Dropout(0.3),
            nn.Linear(512,256), nn.BatchNorm1d(256), nn.ReLU(True), nn.Dropout(0.3),
            nn.Linear(256,num_classes)
        )

    def forward(self, x):  # x: (B,3,N)
        f = self.feat(x)        # (B,F)
        return self.fc(f)       # (B,C) 클래스 로짓 출력


# ------------------------------------------------------------------------
# [PointNet Segmentation Head]
# - 각 포인트마다 클래스 예측 (Semantic Segmentation)
# - global feature를 point feature에 concatenate → shared MLP
# ------------------------------------------------------------------------
class PointNetSeg(nn.Module):
    def __init__(self, num_classes=4, feat_dim=256):
        super().__init__()
        self.feat = PointNetFeat(global_feat=False, feat_dim=feat_dim)  # per-point
        in_ch = feat_dim * 2
        self.mlp = nn.Sequential(
            nn.Conv1d(in_ch, 256,1), nn.BatchNorm1d(256), nn.ReLU(True),
            nn.Conv1d(256, 128,1), nn.BatchNorm1d(128), nn.ReLU(True),
            nn.Conv1d(128, num_classes,1)
        )

    def forward(self, x):  # x: (B,3,N)
        f = self.feat(x)       # (B,2F,N)
        logits = self.mlp(f)   # (B,C,N)
        return logits

In [ ]:
# ------------------------------------------------------------------------
# [📌 Custom Dataset] Sphere vs Cube 분류를 위한 Synthetic PointCloud Dataset
# ------------------------------------------------------------------------
class ClsDataset(Dataset):
    def __init__(self, nitem=400, npts=1024):
        """
        Args:
            nitem: 전체 샘플 개수
            npts: 각 샘플당 포인트 수
        """
        self.n = nitem
        self.npts = npts

    def __len__(self):
        return self.n

    def __getitem__(self, idx):
        if idx % 2 == 0:
            pts = sample_sphere(self.npts)
            y = 0  # Sphere 클래스
        else:
            pts = sample_cube(self.npts)
            y = 1  # Cube 클래스

        # 포인트 정규화: 중심화 후, 최대 반지름 1로 스케일링
        pts = pts - pts.mean(0, keepdims=True)
        s = np.linalg.norm(pts, axis=1).max() + 1e-6
        pts = pts / s

        return torch.from_numpy(pts).float(), torch.tensor(y).long()


# ------------------------------------------------------------------------
# [📌 Dataset/DataLoader 구성]
# ------------------------------------------------------------------------
train_ds = ClsDataset(400, 1024)   # 학습용 데이터셋 (400 샘플)
val_ds   = ClsDataset(100, 1024)   # 검증용 데이터셋 (100 샘플)

train_loader = DataLoader(train_ds, batch_size=32, shuffle=True, drop_last=True)
val_loader   = DataLoader(val_ds, batch_size=32, shuffle=False)


# ------------------------------------------------------------------------
# [📌 PointNet 분류기 초기화]
# ------------------------------------------------------------------------
model = PointNetCls(num_classes=2).to(device)           # 클래스 수: 2 (sphere, cube)
opt   = torch.optim.Adam(model.parameters(), lr=1e-3)   # Adam optimizer
crit  = nn.CrossEntropyLoss()                           # CrossEntropy loss


# ------------------------------------------------------------------------
# [📌 평가 함수: 정확도 계산]
# ------------------------------------------------------------------------
def evaluate(loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for pts, y in loader:
            x = pts.permute(0, 2, 1).to(device)  # (B, 3, N) 형태로 변환
            y = y.to(device)
            logits = model(x)
            pred = logits.argmax(1)
            correct += (pred == y).sum().item()
            total += y.numel()
    return correct / total

In [ ]:
# ------------------------------------------------------------------------
# [📌 간단한 학습 루프] (10 epoch, GPU에서 수초 내 실행)
# ------------------------------------------------------------------------
for epoch in range(10):
    model.train()
    for pts, y in train_loader:
        x = pts.permute(0, 2, 1).to(device)  # PointNet 입력 형태 (B, 3, N)
        y = y.to(device)

        logits = model(x)
        loss = crit(logits, y)

        opt.zero_grad()
        loss.backward()
        opt.step()

    # Epoch별 정확도 출력
    acc = evaluate(val_loader)
    print(f"[Epoch {epoch+1}] val acc: {acc:.3f}")

In [ ]:
import matplotlib.pyplot as plt

# ------------------------------------------------------------------------
# [📌 클래스 이름 정의]
# - 0: sphere, 1: cube
# ------------------------------------------------------------------------
class_names = ["sphere", "cube"]


# ------------------------------------------------------------------------
# [📌 검증 배치에서 샘플 추출 및 예측 수행]
# ------------------------------------------------------------------------
pts_batch, y_batch = next(iter(val_loader))  # pts: (B, N, 3), y: (B,)
x_batch = pts_batch.permute(0, 2, 1).to(device)  # (B, 3, N) 형태로 변환 (PointNet 입력 형태)

with torch.no_grad():
    pred_batch = model(x_batch).argmax(1).cpu().numpy()  # 예측 클래스 인덱스


# ------------------------------------------------------------------------
# [📌 시각화] 최대 8개 샘플을 3D scatter로 시각화
# ------------------------------------------------------------------------
num_vis = min(8, pts_batch.shape[0])  # 최대 8개만 시각화
fig = plt.figure(figsize=(16, 4))     # 2행 4열 서브플롯

for i in range(num_vis):
    pts = pts_batch[i].numpy()                        # (N,3)
    pred_label = class_names[pred_batch[i]]           # 예측 결과 문자열
    gt_label = class_names[y_batch[i].item()]         # 정답 문자열

    ax = fig.add_subplot(2, 4, i+1, projection='3d')  # 2행 4열 중 i+1 번째 subplot
    ax.scatter(pts[:, 0], pts[:, 1], pts[:, 2],
               s=1, c='royalblue', alpha=0.8)         # 파란색 점 시각화

    ax.set_title(f"Pred: {pred_label}\nGT: {gt_label}")  # 제목: 예측 / 정답
    ax.set_axis_off()                                   # 축 숨김

    # 정규화된 포인트클라우드 → 비율 맞춤 (equal aspect)
    mins, maxs = pts.min(0), pts.max(0)
    ctr = (mins + maxs) / 2
    rng = (maxs - mins).max() / 2
    ax.set_xlim(ctr[0] - rng, ctr[0] + rng)
    ax.set_ylim(ctr[1] - rng, ctr[1] + rng)
    ax.set_zlim(ctr[2] - rng, ctr[2] + rng)

# ------------------------------------------------------------------------
# [📌 결과 저장]
# ------------------------------------------------------------------------
plt.tight_layout()
plt.savefig("cls_batch_grid.png", dpi=200)  # 이미지로 저장
plt.close()

print("Saved: cls_batch_grid.png")

In [ ]:
from IPython.display import Image, display
display(Image("cls_batch_grid.png"))

In [ ]:
# ------------------------------------------------------------------------
# [📌 Synthetic indoor scene 생성 함수]
# - 바닥(Floor), 벽(Wall), 박스(Box)로 구성된 포인트클라우드와 라벨 생성
# - 라벨 정의: 0 = floor, 1 = wall, 2 = box
# ------------------------------------------------------------------------

def make_room(n_floor=4000, n_wall=4000, n_box=2000, n_boxes=3, seed=0):
    rng = np.random.default_rng(seed)

    # ------------------------------
    # Floor 생성 (z ≈ 0)
    # ------------------------------
    xy = rng.uniform(-3, 3, size=(n_floor,2))           # X, Y 평면에서 무작위 샘플링
    z  = rng.normal(0, 0.005, size=(n_floor,1))          # Z는 0 주변에 약간의 노이즈
    floor = np.hstack([xy, z])                          # (N,3)
    floor_lab = np.zeros((n_floor,), np.int64)          # 라벨 0 (floor)

    # ------------------------------
    # Wall 생성 (x=±3, y=±3 평면)
    # ------------------------------
    walls = []
    per = n_wall // 4  # 벽 4개로 균등 분배

    # 각 벽에 대해 고정된 축에 ±3 위치 + 다른 축/높이 랜덤
    # x = +3
    y = rng.uniform(-3,3,(per,1)); z = rng.uniform(0,2.5,(per,1))
    x = np.full((per,1), 3.0) + rng.normal(0, 0.01, (per,1))
    walls.append(np.hstack([x, y, z]))

    # x = -3
    y = rng.uniform(-3,3,(per,1)); z = rng.uniform(0,2.5,(per,1))
    x = np.full((per,1), -3.0) + rng.normal(0, 0.01, (per,1))
    walls.append(np.hstack([x, y, z]))

    # y = +3
    x = rng.uniform(-3,3,(per,1)); z = rng.uniform(0,2.5,(per,1))
    y = np.full((per,1), 3.0) + rng.normal(0, 0.01, (per,1))
    walls.append(np.hstack([x, y, z]))

    # y = -3
    x = rng.uniform(-3,3,(per,1)); z = rng.uniform(0,2.5,(per,1))
    y = np.full((per,1), -3.0) + rng.normal(0, 0.01, (per,1))
    walls.append(np.hstack([x, y, z]))

    walls = np.vstack(walls)                            # (n_wall, 3)
    wall_lab = np.ones((walls.shape[0],), np.int64)     # 라벨 1 (wall)

    # ------------------------------
    # Box 객체 여러 개 생성 (n_boxes 개)
    # ------------------------------
    boxes = []
    for bi in range(n_boxes):
        # 중심 위치 무작위
        cx, cy = rng.uniform(-2, 2), rng.uniform(-2, 2)
        # 크기 무작위 (박스마다 다름)
        sx, sy, sz = rng.uniform(0.3,1.0), rng.uniform(0.3,1.0), rng.uniform(0.5,1.2)

        # 박스 표면을 구성하는 포인트 샘플링
        m = n_box // n_boxes  # 박스당 포인트 수
        u  = rng.uniform(-sx/2, sx/2, size=(m,1))
        v  = rng.uniform(-sy/2, sy/2, size=(m,1))
        w  = rng.uniform(0, sz, size=(m,1))

        faces = []
        # top face
        faces.append(np.hstack([u + cx, v + cy, w + 0]))

        # side faces (앞/뒤/좌/우)
        faces.append(np.hstack([np.full_like(u, cx+sx/2), v+cy, w]))
        faces.append(np.hstack([np.full_like(u, cx-sx/2), v+cy, w]))
        faces.append(np.hstack([u+cx, np.full_like(v, cy+sy/2), w]))
        faces.append(np.hstack([u+cx, np.full_like(v, cy-sy/2), w]))

        B = np.vstack(faces) + rng.normal(0, 0.01, size=(len(faces)*m, 3))  # 박스 노이즈 추가
        boxes.append(B)

    boxes = np.vstack(boxes)
    box_lab = np.full((boxes.shape[0],), 2, np.int64)    # 라벨 2 (box)

    # ------------------------------
    # 최종 점군/라벨 병합
    # ------------------------------
    pts = np.vstack([floor, walls, boxes]).astype(np.float32)  # (N,3)
    lbl = np.hstack([floor_lab, wall_lab, box_lab]).astype(np.int64)  # (N,)

    # ------------------------------
    # 정규화: 중심 정렬 + 크기 정규화
    # ------------------------------
    pts = pts - pts.mean(0, keepdims=True)
    s   = (np.linalg.norm(pts, axis=1).max() + 1e-6)
    pts = pts / s

    return pts, lbl

In [ ]:
# ------------------------------------------------------------------------
# [📌 Segmentation Dataset 정의: SegDataset]
# - make_room() 함수를 이용해 synthetic indoor scene을 생성
# - 각 scene에서 일정 개수(npts)의 point + label 추출
# ------------------------------------------------------------------------
class SegDataset(Dataset):
    def __init__(self, n_scenes=200, npts=4096):
        self.n = n_scenes        # 생성할 scene 수
        self.npts = npts         # 각 scene당 샘플링할 포인트 수
        self.scenes = [make_room(seed=i) for i in range(n_scenes)]  # seed 다르게 해서 다양하게 생성

    def __len__(self): return self.n

    def __getitem__(self, idx):
        pts, lbl = self.scenes[idx]  # (N,3), (N,)

        # ------------------------------
        # [🎯 고정 개수 포인트 샘플링 (4096개)]
        # - 포인트 수가 충분하면 무작위 샘플링
        # - 부족하면 중복 허용하여 뽑기
        # ------------------------------
        if pts.shape[0] >= self.npts:
            ids = np.random.choice(pts.shape[0], self.npts, replace=False)
        else:
            ids = np.random.choice(pts.shape[0], self.npts, replace=True)

        p = pts[ids]     # (4096,3)
        y = lbl[ids]     # (4096,)
        return torch.from_numpy(p).float(), torch.from_numpy(y).long()


# ------------------------------------------------------------------------
# [📌 데이터 로더 구성]
# ------------------------------------------------------------------------
train_ds = SegDataset(n_scenes=200, npts=4096)
val_ds   = SegDataset(n_scenes=20,  npts=4096)

train_loader = DataLoader(train_ds, batch_size=8, shuffle=True, drop_last=True)
val_loader   = DataLoader(val_ds,   batch_size=8, shuffle=False)

# ------------------------------------------------------------------------
# [📌 모델 및 Optimizer 정의]
# - PointNet 기반 segmentation 모델 사용 (feat_dim=128)
# - 라벨 클래스 수: 3 (floor=0, wall=1, box=2)
# ------------------------------------------------------------------------
num_classes = 3  # 클래스 수: floor, wall, box

seg_model = PointNetSeg(num_classes=num_classes, feat_dim=128).to(device)

# Adam optimizer + weight decay 추가
opt = torch.optim.Adam(seg_model.parameters(), lr=1e-3, weight_decay=1e-4)

# Cross Entropy Loss
crit = nn.CrossEntropyLoss()


# ------------------------------------------------------------------------
# [📌 평가 함수: seg_eval]
# - 전체 포인트에 대해 정확도 (%) 계산
# ------------------------------------------------------------------------
def seg_eval(loader):
    seg_model.eval()
    tot = 0
    correct = 0
    with torch.no_grad():
        for pts, y in loader:
            x = pts.permute(0, 2, 1).to(device)  # (B,3,N)
            y = y.to(device)                     # (B,N)

            logits = seg_model(x)                # (B,C,N)
            pred = logits.argmax(1)              # (B,N)

            correct += (pred == y).sum().item()
            tot     += y.numel()
    return correct / tot  # 포인트 단위 정확도 (%)

In [ ]:
# ------------------------------------------------------------------------
# [📌 빠른 학습 루프 (~수 초)]
# - 10 에폭 동안 학습 수행
# - 에폭마다 validation point-level 정확도 출력
# ------------------------------------------------------------------------
for ep in range(10):
    seg_model.train()  # 학습 모드 전환

    for pts, y in train_loader:
        # pts: (B, N, 3), y: (B, N)

        # 입력 포인트를 (B,3,N) 형태로 변경하여 모델에 입력
        x = pts.permute(0, 2, 1).to(device)  # (B,3,N)
        y = y.to(device)                     # (B,N)

        # forward pass
        logits = seg_model(x)               # (B, C, N)

        # CrossEntropyLoss 계산 (포인트 단위 다중 클래스 분류)
        loss = crit(logits, y)

        # backward + optimizer step
        opt.zero_grad()
        loss.backward()
        opt.step()

    # ------------------------------
    # [🔍 Validation Accuracy 계산]
    # - point-level 평균 정확도 출력
    # ------------------------------
    acc = seg_eval(val_loader)
    print(f"[Epoch {ep+1}] val point-acc: {acc:.3f}")

In [ ]:
def save_seg(pts, gt_idx, pred_idx,
             class_names, palette,
             fname="seg.png",
             elev=20, azim=60, s=0.5):
    """
    🔸 3D Semantic Segmentation 결과를 시각화하여 PNG 파일로 저장합니다.

    Args:
        pts        : (N, 3) ndarray — 포인트 클라우드 좌표
        gt_idx     : (N,) int — GT 레이블 인덱스
        pred_idx   : (N,) int — 예측 레이블 인덱스
        class_names: 클래스 이름 리스트 (예: ["floor", "wall", "box"])
        palette    : 색상 팔레트 (K,3) RGB 배열
        fname      : 저장할 파일 이름
        elev, azim : 뷰포트 설정 (elevation, azimuth)
        s          : 점 크기 (scatter size)

    Returns:
        acc        : 예측 정확도 (float)
    """

    # -------------------------------------
    # 색상 매핑 (GT / 예측 → RGB로)
    # -------------------------------------
    C_gt   = palette[gt_idx]     # GT 색상
    C_pred = palette[pred_idx]   # 예측 색상

    # -------------------------------------
    # 정확도 계산 (point-level)
    # -------------------------------------
    acc = (gt_idx == pred_idx).mean()

    # -------------------------------------
    # 범례 패치 생성
    # - matplotlib의 Patch 객체를 이용해 색상-클래스 매핑 표현
    # -------------------------------------
    legend_handles = [
        Patch(facecolor=palette[i], edgecolor='k', label=class_names[i])
        for i in range(len(class_names))
    ]

    # -------------------------------------
    # 1행 2열 subplot 생성 (GT vs Prediction)
    # -------------------------------------
    fig = plt.figure(figsize=(10, 4))
    for col, (title, cols) in enumerate(
        [("GT", C_gt), (f"Pred  |  acc={acc*100:.1f}%", C_pred)],
        start=1
    ):
        ax = fig.add_subplot(1, 2, col, projection='3d')
        ax.view_init(elev=elev, azim=azim)
        ax.scatter(pts[:, 0], pts[:, 1], pts[:, 2], s=s, c=cols)
        ax.set_title(title, fontsize=11)
        ax.set_axis_off()

        # 축 비율 맞추기 (equal aspect)
        mins, maxs = pts.min(0), pts.max(0)
        ctr = (mins + maxs) / 2
        rng = (maxs - mins).max() / 2
        ax.set_xlim(ctr[0] - rng, ctr[0] + rng)
        ax.set_ylim(ctr[1] - rng, ctr[1] + rng)
        ax.set_zlim(ctr[2] - rng, ctr[2] + rng)

    # -------------------------------------
    # 범례 추가 (아래 중앙 정렬)
    # -------------------------------------
    fig.legend(
        handles=legend_handles,
        loc="lower center",
        ncol=len(class_names),
        bbox_to_anchor=(0.5, -0.02),
        fontsize=10,
        frameon=False
    )

    # -------------------------------------
    # 그림 저장
    # -------------------------------------
    plt.tight_layout()
    plt.savefig(fname, dpi=200, bbox_inches='tight', pad_inches=0.05)
    # plt.close()  # 필요 시 닫기
    print(f"Saved: {fname}")

    return acc

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.patches import Patch

# -------------------------------
# 클래스 이름과 색상 팔레트 정의
# -------------------------------
# class_names: 레이블 인덱스 0,1,2에 대응하는 이름
# palette: 각 클래스에 대한 RGB 색상 (0~1 스케일)
class_names = ["floor", "wall", "box"]
palette     = np.array([
    [0.4, 0.8, 1.0],   # floor: 밝은 파란색
    [0.9, 0.6, 0.4],   # wall : 주황빛
    [0.5, 1.0, 0.5]    # box  : 초록색
])

# -------------------------------------------------------------
# 검증셋(val_ds)에서 하나의 synthetic scene 샘플을 가져옵니다
# - pts: (N,3) point 좌표
# - y  : (N,) ground-truth label 인덱스
# -------------------------------------------------------------
pts, y = val_ds[0]

# -------------------------------------------------------------
# 모델에 입력 (1개 배치로 차원 확장 + permute to (B,3,N))
# -------------------------------------------------------------
x = pts.unsqueeze(0).permute(0, 2, 1).to(device)  # (1, 3, N)

# -------------------------------------------------------------
# 추론 수행 (logits → argmax → pred)
# -------------------------------------------------------------
with torch.no_grad():
    pred = seg_model(x).argmax(1).squeeze(0).cpu().numpy()  # (N,)

# -------------------------------------------------------------
# 포인트를 numpy로 변환
# -------------------------------------------------------------
pts_np = pts.numpy()  # (N, 3)

# -------------------------------------------------------------
# 시각화 및 정확도 계산
# - GT와 예측 결과를 3D 시각화하여 한 이미지로 저장
# - 범례 포함
# - 정확도(정답률)를 리턴
# -------------------------------------------------------------
acc = save_seg(pts_np,             # 포인트 좌표 (N, 3)
               y.numpy(),          # GT 레이블
               pred,               # 예측 레이블
               class_names,        # 클래스 이름
               palette,            # 색상 팔레트
               fname="seg_gt_pred.png",  # 저장 파일명
               elev=25, azim=45, s=0.6)  # 뷰포트 설정 및 점 크기

print(f"scene accuracy: {acc:.4f}")  # 정확도 출력